In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
dir_processed = "../data/processed"

In [ ]:
matches = pd.read_csv(dir_processed+"matches.csv", index_col=0)

In [ ]:
matches.head()

In [ ]:
def calculate_predicted_score(
    rating1: float,
    rating2: float
) -> float:
    """
    Calculate the predicted score using ELO rating system
    
    Parameters
    ----------
    rating1, rating2: float
        ELO ratings of two players
    
    Returns
    -------
    float
        The predicted score of a player with rating1 against
        a player with rating2 using the ELO rating system
    """
    return 1 / (1 + 10**((rating2 - rating1) / 400))

In [ ]:
def calculate_new_ratings(
    rating_winner: float,
    rating_loser: float,
    predicted_score: float,
    K: float = 32,
) -> (float, float):
    """
    Calculate new elo ratings.

    Parameters
    ----------
    rating_winner, rating_loser: float
        ELO ratings of the winner and loser, respectively.
    predicted_score: float in range [0, 1]
        The expected score of the winner of the match.
    K: float, default 32
        Constant that determines how much the ratings are adjusted.

    Returns
    -------
    new_rating_winner, new_rating_loser: float
        New ELO ratings
    """
    delta_rating = K * (1 - predicted_score)

    new_rating_winner = rating_winner + delta_rating
    new_rating_loser = rating_loser - delta_rating

    return new_rating_winner, new_rating_loser

In [ ]:
# manually test the above functions

for delta in range(-500, 501, 50):
    predicted_score = calculate_predicted_score(delta, 0)
    new_rating_winner, new_rating_loser = calculate_new_ratings(
        delta, 0, predicted_score
    )
    
    print(f'Old winner rating: {delta:3}.')
    print(f'Old loser rating: 0')
    print(f'Predicted score: {predicted_score}')
    print(f'New winner rating: {new_rating_winner}')
    print(f'New loser rating: {new_rating_loser}')
    print()

In [ ]:
def update_ratings(player_ratings, winner_name, loser_name):
    """
    Update ratings based on a single new result.

    If winner or loser is not already in the player_ratings
    dictionary, then a fresh entry with a rating of 1500 is
    created.

    Parameters
    ----------
    player_ratings: Dict[str, float]
        dictionary of player ratings
    winner_name, loser_name: str,
        name of winner and loser

    Returns
    -------
    player_ratings
        updated player ratings
    rating_winner_old
    rating_winner_new
    rating_loser_old
    rating_loser_new
    predicted_score
    """
    if winner_name not in player_ratings:
        player_ratings[winner_name] = 1500
    rating_winner_old = player_ratings[winner_name]

    if loser_name not in player_ratings:
        player_ratings[loser_name] = 1500
    rating_loser_old = player_ratings[loser_name]

    predicted_score = calculate_predicted_score(rating_winner_old, rating_loser_old)

    rating_winner_new, rating_loser_new = calculate_new_ratings(
        rating_winner_old, rating_loser_old, predicted_score
    )
    
    player_ratings[winner_name] = rating_winner_new
    player_ratings[loser_name] = rating_loser_new

    return (
        player_ratings,
        rating_winner_old,
        rating_winner_new,
        rating_loser_old,
        rating_loser_new,
        predicted_score,
    )

In [ ]:
# manually test update_ratings

test_ratings = {'a': 1000, 'b': 900}

test_ratings, _, _, _, _, _ = update_ratings(test_ratings, 'a', 'b')
test_ratings, _, _, _, _, _ = update_ratings(test_ratings, 'c', 'd')
test_ratings, _, _, _, _, _ = update_ratings(test_ratings, 'd', 'c')

test_ratings

In [ ]:
def calculate_elo(matches: pd.DataFrame, player_ratings={}) -> pd.DataFrame:
    """
    Calculate elo ratings for all players and match history from
    the matches input.

    Parameters
    ----------
    matches: pd.DataFrame
        dataframe of match history
    player_ratings: Dict[str, float]
        dictionary of players' elo ratings

    Returns
    -------
    player_ratings: Dict[str, float]
        updated player_ratings
    pd.DataFrame
        copy of matches dataframe with new columns for:
        * rating_winner_old
        * rating_winner_new
        * rating_loser_old
        * rating_loser_new
        * predicted_score
    """
    ratings_winner_old = []
    ratings_winner_new = []
    ratings_loser_old = []
    ratings_loser_new = []
    predicted_scores = []

    df = matches.copy()

    for _, row in tqdm(df.iterrows()):
        winner_name = row.winner_name
        loser_name = row.loser_name

        (
            player_ratings,
            rating_winner_old,
            rating_winner_new,
            rating_loser_old,
            rating_loser_new,
            predicted_score,
        ) = update_ratings(player_ratings, winner_name, loser_name)
        
        ratings_winner_old.append(rating_winner_old)
        ratings_winner_new.append(rating_winner_new)
        ratings_loser_old.append(rating_loser_old)
        ratings_loser_new.append(rating_loser_new)
        predicted_scores.append(predicted_score)
    
    df['rating_winner_old'] = ratings_winner_old
    df['rating_winner_new'] = ratings_winner_new
    df['rating_loser_old'] = ratings_loser_old
    df['rating_loser_new'] = ratings_loser_new
    df['predicted_score'] = predicted_scores
    
    return player_ratings, df

In [ ]:
player_ratings, matches = calculate_elo(matches)
matches.head()

In [ ]:
def view_player_history(df: pd.DataFrame, player: str) -> None:
    """
    View all results of a player.
    
    Prints the following for all games that player played in:
    * name of winner
    * name of loser
    * winners elo rating (before the match)
    * losers elo rating (before the match)
    * winners seed in the tournament
    * losers seed in the tournament
    * predicted score from elo ratings for the match
    
    Parameters
    ----------
    df
        dataframe of matches as outputted by calculate_elo
    player: str
        name of the player
    
    Returns
    -------
    None
    """
    indices = (df.winner_name == player) | (df.loser_name == player)
    df_player = df[indices]
    
    for _,row in df_player.iterrows():
        w = row.winner_name
        wr = row.rating_winner_old
        ws = row.winner_seed
        l = row.loser_name
        lr = row.rating_loser_old
        ls = row.loser_seed
        pred = row.predicted_score
        print(f'{w[0:10]:10} beat {l[0:10]:10} {wr:.0f} vs {lr:.0f}   {ws:3} vs {ls:3}   {pred:.2f}')

In [ ]:
view_player_history(df, 'Ramy Ashour')

In [ ]:
def evaluate_calibration(df_input: pd.DataFrame, N: int = 2) -> pd.Series:
    """
    Evaluate how well calibrated the ELO ratings are.

    Parameters
    ----------
    df_input
        Dataframe as outputted by calculate_elo
    N: int
        Number of times each bucket of size 0.1 is broken up.
        See the index of the returned pd.Series for an example

    Returns
    -------
    pd.Series
        * index is predicted score, rounded to nearest 0.1/N. For
          example, if N=2, then rounded to nearest 0.05, so index is
          0.5, 0.55, 0.6,...,0.95, 1
        * values are the average true score of matches whose predicted
        score is in that bucket
    """
    df = df_input.copy()

    df["predicted_score_better_player"] = df.predicted_score.apply(
        lambda x: round(N * x, 1) / N if x > 0.5 else 1 - round(N * x, 1) / N
    )

    df["true_score_better_player"] = df.predicted_score.apply(
        lambda x: 1 if x > 0.5 else 0
    )

    return (
        df
        .groupby("predicted_score_better_player")
        .agg({"true_score_better_player": ["count", "mean"]})
    )

In [ ]:
evaluate_calibration(df)